# 还原模型(基于单隐层神经网络的手写数字识别)

# 载入数据

In [1]:
import tensorflow as tf

#导入Tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data

#读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#定义全连接层函数
def fcn_layer(inputs,
              input_dim,
              output_dim,
              activation=None):
    
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1)) #以截断正态分布的随机数初始化W
    
    b = tf.Variable(tf.zeros([output_dim])) #以0初始化b
    
    XWb = tf.matmul(inputs, W) + b #建立表达式: inputs * W + b
    
    if activation is None: #默认有使用激活函数
        outputs = XWb
    else: 
        outputs = activation(XWb) # 若传入激活函数，则用其对输出结果进行变换
    
    return outputs

# 构建输入层

In [3]:
x = tf.placeholder(tf.float32, [None, 784], name="X")

# 构建隐藏层

In [4]:
#隐藏层神经元数量256
h1 = fcn_layer(inputs=x,
               input_dim=784,
               output_dim=256,
               activation=tf.nn.relu)

# 构建输出层 

In [5]:
forward = fcn_layer(inputs=h1,
                    input_dim=256,
                    output_dim=10,
                    activation=None)
pred = tf.nn.softmax(forward)

# 训练模型

In [6]:
#定义标签数据占位符
y = tf.placeholder(tf.float32, [None, 10], name="Y")

# 定义损失函数

In [7]:
#交叉熵
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=1380
                                                                       , labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# 设置训练参数

In [11]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

# 选择优化器

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

# 定义准确率

In [13]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 还原模型

# 设置目录

In [14]:
# 必须指定为模型文件的存放目录
ckpt_dir = "./ckpt_dir/"

# 读取模型

In [16]:
saver = tf.train.Saver()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

ckpt = tf.train.get_checkpoint_state(ckpt_dir)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path) #从已保存的模型中读取参数
    print("restore model from "+ckpt.model_checkpoint_path)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./ckpt_dir/mnist_h256_model.ckpt
restore model from ./ckpt_dir/mnist_h256_model.ckpt


# 输出模型准确率

In [17]:
print("Accuracy:", accuracy.eval(session=sess,
                                 feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Accuracy: 0.9707


# 评估模型

In [ ]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print("Test Accuracy:", accu_test)

# 进行预测

In [ ]:
#由于pred预测结果是one-hot编码格式，所以需要转换为0~9数字。
prediction_result=sess.run(tf.argmax(pred,1), feed_dict={x: mnist.test.images})

In [ ]:
#查看预测结果中的前10项
prediction_result[0:10]

# 定义可视化函数

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_images_labels_prediction(images,     #图像列表
                                  labels,     #标签列表
                                  prediction, #预测值列表
                                  index,      #从第index个开始显示
                                  num=10):    #缺省一次显示10幅
    fig = plt.gcf() #获取当前图表,Get Current Figure
    fig.set_size_inches(10,12) #1英寸等于2.54cm
    if num > 25:
        num = 25                #最多显示25个子图
    for i in range(0, num):
        ax = plt.subplot(5,5, i+1) #获取当前要处理的子图
        ax.imshow(np.reshape(images[index],(28, 28)),cmap='binary')
        title = "label=" + str(np.argmax(labels[index])) #构建该图上要显示的title
        if len(prediction)>0:
            title += ",predict=" + str(prediction[index])
        ax.set_title(title,fontsize=10) #显示图上的title信息
        ax.set_xticks([]); #不显示坐标轴
        ax.set_yticks([])
        index += 1
    plt.show()

In [ ]:
plot_images_labels_prediction(mnist.test.images,
                              mnist.test.labels,
                              prediction_result,0,25)

# 找出预测错误

In [ ]:
import numpy as np
compare_lists = prediction_result==np.argmax(mnist.test.labels,1) 
print(compare_lists)

In [ ]:
err_lists = [i for i in range(len(compare_lists)) if compare_lists[i]==False]
print(err_lists, len(err_lists))

# 定义一个输出错误分类的函数

In [ ]:
def print_predict_errs(labels,      #标签列表
                       prediction): #预测值列表
    count = 0
    compare_lists = (prediction==np.argmax(labels,1))
    err_lists = [i for i in range(len(compare_lists)) if compare_lists[i]==False]
    for x in err_lists:
        print("index="+str(x)+ 
              " 标签值=",np.argmax(labels[x]),
              "预测值=",prediction[x])
        count = count + 1
    print("总计:"+str(count))

In [ ]:
print_predict_errs(labels=mnist.test.labels,
                   prediction=prediction_result)